# Learning Lab: Interface

This Learning Lab explores the network interfaces of a network device.

Pre-requisite Learning Lab: [Inventory](Inventory.ipynb)

|Keyword|Definition|
|--|--|
|Network Interface|Connector from one network device to a network interface on one or more other network devices.|
|Network Device|Routers, switches, etc.|
|Controller|Service that intermediates between client applications and network devices. E.g. ODL, COSC|
|ODL|OpenDayLight|
|COSC|Cisco Open Source Controller, enhanced ODL|

## Table of Contents

* [Table of Contents](#Table-of-Contents)
* [Prologue](#Prologue)
    * [Import Learning Lab](#Import-Learning-Lab)
    * [Verify Controller](#Verify-Controller)
    * [Sample Network Device](#Sample-Network-Device)
* [Introduction](#Introduction)
* [List of Network Interfaces](#List-of-Network-Interfaces)
* [Management Interface](#Management-Interface)
* [Interface Configuration](#Interface-Configuration)
    * [XML](#Interface-Configuration:-XML)
        * [For *all* network interfaces](#For-all-network-interfaces)
        * [For *one* network interface](#For-one-network-interface)
    * [JSON](#Interface-Configuration:-JSON)
    * [Python type `InterfaceConfiguration`](#Python-type-InterfaceConfiguration)
* [Interface Properties](#Interface-Properties)
    * [XML](#Interface-Properties:-XML)
    * [JSON](#Interface-Properties:-JSON)
    * [Python type `InterfaceProperties`](#Python-type-InterfaceProperties)
* [Conclusion](#Conclusion)
* [HTTP Reference](#HTTP-Reference)
    * [RPC `interface_configuration_xml(device)`](#RPC-interface_configuration_xml:-multiple)
    * [RPC `interface_configuration_xml(device, interface)`](#RPC-interface_configuration_xml:-single)
    * [RPC `interface_configuration_json(device)`](#RPC-interface_configuration_json:-multiple)
    * [RPC `interface_configuration_json(device, interface)`](#RPC-interface_configuration_json:-single)
    * [RPC `interface_properties_xml(device)`](#RPC-interface_properties_xml:-multiple)
    * [RPC `interface_properties_xml(device, interface)`](#RPC-interface_properties_xml:-single)
    * [RPC `interface_properties_json(device)`](#RPC-interface_properties_json:-multiple)
    * [RPC `interface_properties_json(device, interface)`](#RPC-interface_properties_json:-single)

## Prologue

### Import Learning Lab

If the Learning Lab team have provided you with either:
* a Python Interactive Shell, or
* an IPython Notebook
  
then please proceed. Otherwise, please follow the instructions on web page: [How to set up your computer](HowToSetUpYourComputer.ipynb)

Import the Python module named `learning_lab`. 

In [1]:
import learning_lab

### Sample Network Device

We need one network device to explore. Any network device is satisfactory, provided it is connected to the Controller and has multiple network interfaces.
  
The following code selects a suitable network device from the inventory using convenience function `sample_device()`. 

In [2]:
%run 01_inventory_mount
from basics.sample import sample_device

device_name = 'xrvr-1'

Python Library Documentation: function inventory_mount in module basics.inventory

inventory_mount(device_name, device_address, device_port, device_username, device_password)
    Add the specified network device to the inventory of the Controller.

All configured devices are mounted.
xrvr-1
MgmtEth0/0/CPU0/0
MgmtEth0/0/CPU0/0
GigabitEthernet0/0/0/1


Note that we only require the *name* of the sample network device. 

In [3]:
device_name

'xrvr-1'

## Introduction

In this lesson, we will:
1. List the names of network interfaces on a single network device. 
2. Determine which network interface the Controller is using to manage the network device. 
3. For each network interface, obtain: 
    * configuration
    * properties
4. Configuration and properties will be obtained:
    * in bulk - all network interfaces
    * individually - a single network interface, specified by name
5. The data formats will be:
    * XML - demonstration of transformation to HTML using XSLT
    * JSON - indexing deep into a complex document
    * Python - types `InterfaceConfiguration` and `InterfaceProperties`    

## List of Network Interfaces

List the network interfaces of the selected network device, using function:
* `interface_names(device_name)`.

In [4]:
from basics.interface import interface_names

interface_names(device_name)

['MgmtEth0/0/CPU0/0', 'GigabitEthernet0/0/0/1', 'GigabitEthernet0/0/0/0']

## Management Interface

Determine which network interface the Controller is using to manage the network device, using function:
* `management_interface(device_name)`

In [5]:
from basics.interface import management_interface

management_interface(device_name)

'MgmtEth0/0/CPU0/0'

Each network interface belongs to one of the following two categories:
* management interface
* data interface
  
Normal network traffic flows between data interfaces. A management interface is for monitoring and modifying the data interfaces on the same network device. For example, you can shutdown/startup a data interface via a management interface.
  
Note: if you shutdown a management interface then the connection terminates and you will not be able to send it the startup command.

## Interface Configuration

### Interface Configuration: XML

#### For *all* network interfaces

Obtain the configuration of all network interfaces on one network device, using function:
* `interface_configuration_xml(device_name)`
  
The XML representation of data is popular in client applications. The Controller is capable of representing the configuration data in either XML or JSON format. The HTTP Reference shows the actual  [XML](#RPC-interface_configuration_xml:-multiple) and [JSON](#RPC-interface_configuration_json:-multiple) documents returned from the Controller.

In [6]:
from basics.interface import interface_configuration_xml

xml = interface_configuration_xml(device_name)

Additional functions are used to transform the XML into a HTML table to make it pleasant for humans to read. The implementation uses XSLT.

In [7]:
from basics.interface import interface_configuration_to_html
from IPython.core.display import HTML

HTML(interface_configuration_to_html(xml))

#### For *one* network interface

A single network interface can be specified by name as the second parameter to function:
* `interface_configuration_xml(device_name, interface_name)`
  
A single network interface is chosen, randomly, from the list of interface names.

In [8]:
import random

interface_name = random.choice(interface_names(device_name))

The XML representation is obtained...

In [9]:
xml = interface_configuration_xml(device_name, interface_name)

... and transformed to HTML. We expect the HTML table to contain exactly one row of data, identical to one row in the table above.

In [10]:
HTML(interface_configuration_to_html(xml))

### Interface Configuration: JSON

XML is ideal for transformation to another markup language, such as HTML, as shown above. JSON is an alternative to XML that is popular. Network interface configuration is available in JSON from functions:
* `interface_configuration_json(device_name)`
* `interface_configuration_json(device_name, interface_name)`

In [11]:
from basics.interface import interface_configuration_json

json = interface_configuration_json(device_name, interface_name)

The JSON for a single network interface is reasonably small.

In [12]:
json

{u'interface-configuration': [{u'Cisco-IOS-XR-ipv4-io-cfg:ipv4-network': {u'addresses': {u'primary': {u'address': u'10.0.0.6',
      u'netmask': u'255.255.255.252'}}},
   u'active': u'act',
   u'description': u'to iosxrv-2',
   u'interface-name': u'GigabitEthernet0/0/0/0'}]}

The JSON for *all* network interfaces (of one network device).

In [13]:
json = interface_configuration_json(device_name)

This JSON data structure is shown in the [HTTP Reference](#RPC-interface_configuration_json:-multiple) due to its complexity.

Typically usage of JSON is to index into the data structure. This example accesses the interface name and description. 

In [14]:
[(
    item[u'interface-name'], 
    item[u'description']
 ) 
 for item in json
     [u'interface-configurations']
     [u'interface-configuration']]

[(u'GigabitEthernet0/0/0/0', u'to iosxrv-2'),
 (u'MgmtEth0/0/CPU0/0', u'OOB Management'),
 (u'Loopback0', u'Loopback'),
 (u'GigabitEthernet0/0/0/1', u'to iosxrv-3')]

It is more difficult to access the IP address because the path depends on IPv4 versus IPv6. Note that in XML the IP address is accessed using a XPATH expression with a wildcard matching either IPv4 or IPv6.

In [15]:
xml.findtext('//{*}primary/{*}address')

'10.0.0.6'

### Python type `InterfaceConfiguration`

Configuration of network interfaces is available as a Python [`named tuple`](https://foo.cs.ucsb.edu/8wiki/index.php/Python:_Named_Tuples). The functions are:
* `interface_configuration_tuple(device_name)` 
* `interface_configuration_tuple(device_name, interface_name)` 

In [16]:
from basics.interface import interface_configuration_tuple

tuple = interface_configuration_tuple(device_name, interface_name)

The type of the named tuple is a class in Python.

In [17]:
type(tuple)

basics.interface.InterfaceConfiguration

A tuple provides access to the fields in a language that is very natural. Several fields are accessed to demonstrate.

In [18]:
tuple.name

'GigabitEthernet0/0/0/0'

In [19]:
tuple.address

'10.0.0.6'

In [20]:
tuple.netmask

'255.255.255.252'

In [21]:
tuple.shutdown

False

The value of field `shutdown`, above, is implied by the existence or absence of a specific element in XML (or field in JSON). For this reason, the Python tuple is more convenient to use in a client application than a test for the presence of fields in XML and JSON.

In [22]:
json = interface_configuration_json(device_name, interface_name)
shutdown = 'shutdown' in json['interface-configuration'][0]
assert shutdown == tuple.shutdown

List all field names of the Python class.

In [23]:
tuple.__dict__.keys()

['name',
 'description',
 'shutdown',
 'address',
 'netmask',
 'packet_filter_outbound',
 'packet_filter_inbound',
 'active']

Note that the tuple field names are not strings, as they are in JSON. Here is the access of field `description` in JSON.

In [24]:
json['interface-configuration'][0]['description']

u'to iosxrv-2'

The JSON field access, above, can only be verified at run-time, whereas the tuple field access, below, can be checked by the editor and can participate in *code completion*, because the field is pre-defined. This is another way the Python tuple adds value to the underlying XML or JSON (from which the tuple is built).

In [25]:
tuple.description

'to iosxrv-2'

All examples of the Python tuple, above, have been for a single network interface. The configuration of multiple network interfaces (of one network device) is available as a list of tuples.

In [26]:
bulk = interface_configuration_tuple(device_name)

[(tuple.name, tuple.address) for tuple in bulk]

[('GigabitEthernet0/0/0/0', '10.0.0.6'),
 ('MgmtEth0/0/CPU0/0', '172.16.1.88'),
 ('Loopback0', '192.168.0.1'),
 ('GigabitEthernet0/0/0/1', '10.0.128.1')]

## Interface Properties

The Controller is capable of representing network interface properties in either XML or JSON format. The HTTP Reference shows the actual [XML](#RPC-interface_properties_xml:-multiple) and [JSON](#RPC-interface_properties_json:-multiple) documents returned from the Controller.

### Interface Properties: XML

Obtain the properties of all network interfaces of one network device, using function:
* `interface_properties_xml(device_name)`
  

In [27]:
from basics.interface import interface_properties_xml

xml = interface_properties_xml(device_name)

Additional functions are employed to transform the XML into two HTML tables. The reason for *two* HTML tables is simply to fit all the properties into the width of this document. The XML is transformed to HTML using a XSLT stylesheet.

In [28]:
from basics.interface import interface_properties_to_html

HTML(interface_properties_to_html(xml))

A single network interface can be specified by name as the second parameter to function:
* `interface_properties_xml(device_name, interface_name)`
  
The specific interface was [chosen randomly](#For-one-network-interface). 

In [29]:
xml = interface_properties_xml(device_name, interface_name)

The XML document is displayed in the [HTTP Reference](#RPC-interface_properties_xml:-single). The same XSLT stylesheet used above can be applied to the XML to produce HTML tables. We expect the two HTML tables to contain exactly one row of data, identical to one row in each of the two tables above.

In [30]:
HTML(interface_properties_to_html(xml))

### Interface Properties: JSON

The interface properties are returned in JSON representation from function:
* `interface_properties_json(device_name)`

The [HTTP Reference](#RPC-interface_properties_json:-multiple) shows the actual JSON document returned from the Controller.

The sample code below extracts three of the many properties available for each network interface: 
* interface-name
* type
* encapsulation.

In [31]:

from basics.interface import interface_properties_json

[(
    item[u'interface-name'], 
    item[u'type'], 
    item[u'encapsulation']
 ) 
 for item in interface_properties_json(device_name)
     [u'interface-properties']
     [u'data-nodes']
     [u'data-node']
     [0]
     [u'system-view']
     [u'interfaces']
     [u'interface']
]

[(u'MgmtEth0/0/CPU0/0', u'IFT_ETHERNET', u'ether'),
 (u'Null0', u'IFT_NULL', u'null'),
 (u'FINT0/0/CPU0', u'IFT_FINT_INTF', u'fint_base'),
 (u'Loopback0', u'IFT_LOOPBACK', u'loopback'),
 (u'GigabitEthernet0/0/0/1', u'IFT_GETHERNET', u'ether'),
 (u'GigabitEthernet0/0/0/0', u'IFT_GETHERNET', u'ether'),
 (u'nV-Loopback0', u'IFT_NV_LOOPBACK', u'nv_loopback')]

In [32]:
json = interface_properties_json(device_name, interface_name)

The JSON document is displayed in the [HTTP Reference](#RPC-interface_properties_json:-single). 
  
The property names and values can be accessed directly from the JSON structure.

In [33]:
[(name, value) for (name,value) in json[u'interface'][0].iteritems()]

[(u'encapsulation-type-string', u'ARPA'),
 (u'sub-interface-mtu-overhead', 0),
 (u'interface-name', u'GigabitEthernet0/0/0/0'),
 (u'state', u'im-state-up'),
 (u'actual-state', u'im-state-up'),
 (u'actual-line-state', u'im-state-up'),
 (u'l2-transport', False),
 (u'mtu', 1514),
 (u'line-state', u'im-state-up'),
 (u'bandwidth', 1000000),
 (u'interface', u'GigabitEthernet0/0/0/0'),
 (u'encapsulation', u'ether'),
 (u'type', u'IFT_GETHERNET')]

### Python type `InterfaceProperties`

The properties of a network interface are available as a Python [`named tuple`](https://foo.cs.ucsb.edu/8wiki/index.php/Python:_Named_Tuples). The functions are:
* interface_properties_tuple(device_name)
* interface_properties_tuple(device_name, interface_name)

In [34]:
from basics.interface import interface_properties_tuple

tuple = interface_properties_tuple(device_name, interface_name)

The type of the tuple is a class in Python.

In [35]:
type(tuple)

basics.interface.InterfaceProperties

List all fields of the Python class.

In [36]:
tuple.__dict__.keys()

['name',
 'type',
 'bandwidth',
 'encapsulation',
 'encapsulation_type',
 'state',
 'line_state',
 'actual_state',
 'actual_line_state',
 'l2_transport',
 'mtu',
 'sub_interface_mtu_overhead']

A tuple provides access to the fields in a language that is very natural. Several fields are accessed to demonstrate. Note that the property values below have different types: string, boolean and integer.

In [37]:
tuple.name

u'GigabitEthernet0/0/0/0'

In [38]:
tuple.l2_transport

False

In [39]:
tuple.mtu

1514

All examples of the Python tuple, above, have been for a single network interface. The properties of multiple network interfaces (of one network device) are available as a list of tuples.

In [40]:
bulk = interface_properties_tuple(device_name)

[(tuple.name, tuple.state) for tuple in bulk]

[(u'MgmtEth0/0/CPU0/0', u'im-state-up'),
 (u'Null0', u'im-state-up'),
 (u'FINT0/0/CPU0', u'im-state-up'),
 (u'Loopback0', u'im-state-up'),
 (u'GigabitEthernet0/0/0/1', u'im-state-up'),
 (u'GigabitEthernet0/0/0/0', u'im-state-up'),
 (u'nV-Loopback0', u'im-state-up')]

## Conclusion

In this lesson we:
1. Listed the names of network interfaces on a single network device. 
2. Determined which network interface the Controller is using to manage the network device. 
3. For each network interface we obtained: 
    * configuration
    * properties
4. The configuration and properties of network interfaces are available, per network device:
    * in bulk - all network interfaces
    * individually - a single network interface, specified by name
5. The data formats used were:
    * XML - demonstration of transformation to HTML using XSLT
    * JSON - indexing deep into a complex document
    * Python - types `InterfaceConfiguration` and `InterfaceProperties`

## HTTP Reference

The HTTP request and response are shown for each remote procedure call (RPC) to the Controller.
  
See the Learning Lab document [HTTP](Http.ipynb) for a demonstration of how to use the tables below.

### RPC interface_configuration_xml: multiple

Function:
* `interface_configuration_xml(device_name)`

invokes function:
* `interface_configuration_xml_http(device_name)`

which returns the HTTP request and response, as shown in the table below.
  
The response from the Controller is very large because it describes multiple network interfaces. The next RPC is for a single network interface and is thus shorter. To help you navigate this document, here is a link to the [next section](#RPC-interface_configuration_xml:-single) and the [TOC](#Table-of-Contents).

In [41]:
from basics.interface import interface_configuration_xml_http
from basics.http import http_to_html

HTML(http_to_html(interface_configuration_xml_http(device_name)))

### RPC interface_configuration_xml: single

The function:
* `interface_configuration_xml(device_name, interface_name)`

invokes function:
* `interface_configuration_xml_http(device_name, interface_name)`

which returns the HTTP request and response, as shown in the table below. The specific interface was [chosen randomly](#For-one-network-interface). 
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_configuration_xml:-multiple), [next section](#RPC-interface_configuration_json:-multiple) and the [TOC](#Table-of-Contents).

In [42]:
http = interface_configuration_xml_http(device_name, interface_name)
HTML(http_to_html(http))

### RPC interface_configuration_json: multiple

Function:
* `interface_configuration_json(device_name)`

invokes function:
* `interface_configuration_json_http(device_name)`

which returns the HTTP request and response, as shown in the table below.
  
The response from the Controller is very large because it describes multiple network interfaces. The next RPC is for a single network interface and is thus shorter. To help you navigate this document, here is a link to the [previous section](#RPC-interface_configuration_xml:-single), [next section](#RPC-interface_configuration_json:-single) and the [TOC](#Table-of-Contents).

In [43]:
from basics.interface import interface_configuration_json_http

HTML(http_to_html(interface_configuration_json_http(device_name)))

### RPC interface_configuration_json: single

The function:
* `interface_configuration_json(device_name, interface_name)`

invokes function:
* `interface_configuration_json_http(device_name, interface_name)`

which returns the HTTP request and response, as shown in the table below. The specific interface was [chosen randomly](#For-one-network-interface). 
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_configuration_json:-multiple), [next section](#RPC-interface_properties_xml:-multiple) and the [TOC](#Table-of-Contents).

In [44]:
HTML(http_to_html(interface_configuration_json_http(device_name, interface_name)))

### RPC interface_properties_xml: multiple

The function:
* `interface_properties_xml(device_name)`

invokes function:
* `interface_properties_xml_http(device_name)`

which returns the HTTP request and response, as shown in the table below.
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_configuration_json:-single), [next section](#RPC-interface_properties_xml:-single) and the [TOC](#Table-of-Contents).

In [45]:
from basics.interface import interface_properties_xml_http

HTML(http_to_html(interface_properties_xml_http(device_name)))

### RPC interface_properties_xml: single

The function:
* `interface_properties_json(device_name, interface_name)`

invokes function:
* `interface_properties_json_http(device_name, interface_name)`

which returns the HTTP request and response, as shown in the table below. The specific interface was [chosen randomly](#For-one-network-interface). 
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_properties_xml:-multiple), [next section](#RPC-interface_properties_json:-multiple) and the [TOC](#Table-of-Contents).

In [46]:
HTML(http_to_html(interface_properties_xml_http(device_name, interface_name)))

### RPC interface_properties_json: multiple

The function:
* `interface_properties_json(device_name)`

invokes function:
* `interface_properties_json_http(device_name)`

which returns the HTTP request and response, as shown in the table below.
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_properties_xml:-single), [next section](#RPC-interface_properties_json:-single) and the [TOC](#Table-of-Contents).

In [47]:
from basics.interface import interface_properties_json_http

HTML(http_to_html(interface_properties_json_http(device_name)))

### RPC interface_properties_json: single

The function:
* `interface_properties_json(device_name, interface_name)`

invokes function:
* `interface_properties_json_http(device_name, interface_name)`

which returns the HTTP request and response, as shown in the table below. The specific interface was [chosen randomly](#For-one-network-interface). 
  
To help you navigate around the large tables in this document, here are links to the [previous section](#RPC-interface_properties_json:-multiple) and the [TOC](#Table-of-Contents).

In [48]:
HTML(http_to_html(interface_properties_json_http(device_name, interface_name)))